## Types of Text Summarization Methods

![Imgur](https://i.imgur.com/J5KyMBJ.png)


# Import Packages 


In [10]:
import numpy as np # linear algebra
import spacy
nlp = spacy.load('en_core_web_sm')
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from IPython.display import display
import base64
import string
import re
from collections import Counter
from time import time
from nltk.corpus import stopwords
import nltk
import heapq
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
%matplotlib inline

stopwords = stopwords.words('english')
sns.set_context('notebook')

# Import Dataset 


In [46]:
reviews = pd.read_csv("../data/wine_reviews.csv", usecols =['points', 'title', 'description'], encoding='latin1')
reviews = reviews.dropna()
reviews.head(15)

,description,points,title
0,"Aromas include tropical fruit, broom, brimston...",87,Nicosia 2013 VulkÃ Bianco (Etna)
1,"This is ripe and fruity, a wine that is smooth...",87,Quinta dos Avidagos 2011 Avidagos Red (Douro)
2,"Tart and snappy, the flavors of lime flesh and...",87,Rainstorm 2013 Pinot Gris (Willamette Valley)
3,"Pineapple rind, lemon pith and orange blossom ...",87,St. Julian 2013 Reserve Late Harvest Riesling ...
4,"Much like the regular bottling from 2012, this...",87,Sweet Cheeks 2012 Vintner's Reserve Wild Child...
5,Blackberry and raspberry aromas show a typical...,87,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...
6,"Here's a bright, informal red that opens with ...",87,Terre di Giurfo 2013 Belsito Frappato (Vittoria)
7,This dry and restrained wine offers spice in p...,87,Trimbach 2012 Gewurztraminer (Alsace)
8,Savory dried thyme notes accent sunnier flavor...,87,Heinz Eifel 2013 Shine GewÃ¼rztraminer (Rheinh...
9,This has great depth of flavor with its fresh ...,87,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...


# Text preprocessing

In [ ]:
!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')
def normalize_text(text):
    tm1 = re.sub('<pre>.*?</pre>', '', text, flags=re.DOTALL)
    tm2 = re.sub('<code>.*?</code>', '', tm1, flags=re.DOTALL)
    tm3 = re.sub('<[^>]+>©', '', tm1, flags=re.DOTALL)
    return tm3.replace("\n", "")

In [ ]:

reviews['description_Cleaned_1'] = reviews['description'].apply(normalize_text)

In [ ]:
print('Before normalizing text-----\n')
print(reviews['description'][2])
print('\nAfter normalizing text-----\n')
print(reviews['description_Cleaned_1'][2])

Ваши методы, подумайте до конца ли вы почистили датасет

In [ ]:
punctuations = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~©'
# Define function to cleanup text by removing personal pronouns, stopwords, and puncuation
def cleanup_text(docs, logging=False):
    texts = []
    doc = nlp(docs, disable=['parser', 'ner'])
    tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
    tokens = [tok for tok in tokens if tok not in stopwords and tok not in punctuations]
    tokens = ' '.join(tokens)
    texts.append(tokens)
    return pd.Series(texts)

reviews['Description_Cleaned'] = reviews['description_Cleaned_1'].apply(lambda x: cleanup_text(x, False))

In [ ]:
print('Reviews description with punctuatin and stopwords---\n')
print(reviews['description_Cleaned_1'][0])
print('\nReviews description after removing punctuation and stopwrods---\n')
print(reviews['Description_Cleaned'][0])

# Distribution of Points

# Analyze reviews description

Pipeline for summary using spacy or gensim:

### 1 Convert Paragraphs to Sentences

### 2 Text Preprocessing

### 3 Tokenizing the Sentences

### 4 Find Weighted Frequency of Occurrence

### 5 Replace Words by Weighted Frequency in Original Sentences

### 6 Sort Sentences in Descending Order of Sum

In [ ]:
def generate_summary(text_without_removing_dot, cleaned_text):
    sample_text = ...
    doc = nlp(sample_text)
    sentence_list=[]
    for idx, sentence in enumerate(doc.sents): 
        sentence_list.append(re.sub(r'[^\w\s]','',str(sentence)))

    stopwords = nltk.corpus.stopwords.words('english')

    word_frequencies = {}  
    ...
    maximum_frequncy = max(word_frequencies.values())


    sentence_scores = {}  
    ...

    summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

    summary = ' '.join(summary_sentences)
    print("Original Text:\n")
    print(text_without_removing_dot)
    print('\n\nSummarized text:\n')
    print(summary)  

In [ ]:
generate_summary(reviews['description_Cleaned_1'][8], reviews['Description_Cleaned'][8])

In [ ]:
generate_summary(reviews['description_Cleaned_1'][100], reviews['Description_Cleaned'][100])

In [ ]:
generate_summary(reviews['description_Cleaned_1'][500], reviews['Description_Cleaned'][500])